# Data Analysis for Stock Simulation

This notebook compiles statistics on the performance of each trading strategy as well as overall performance versus changes in the asset price. It:

- Loads the CSV files (`price_data.csv` and `agent_wealth.csv`) generated during simulation
- Reconstructs agent strategy assignments (based on our simulation’s round-robin order for the first 100 agents, plus special arbitrage agents)
- Computes summary statistics (mean, median, standard deviation, min, max) for final wealth by strategy
- Plots overall asset price vs. average agent wealth over time
- Examines the correlation between price changes and wealth changes
and finally produces box plots of the final wealth distribution per strategy.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read the CSV files generated by the simulation
price_data = pd.read_csv('Data/price_data.csv', index_col=0)
wealth_data = pd.read_csv('Data/agent_wealth.csv', index_col=0)

# Preview the imported data
print("Price Data:")
print(price_data.head())

print("\nWealth Data (agents over ticks):")
print(wealth_data.head())

## Agent Strategy Assignment

In the simulation, agents were assigned to strategies as follows:

- **Agents 0–99** (assuming 100 agents) are assigned in a round-robin order to:
  - TrendFollowingStrategy
  - MeanReversionStrategy
  - MomentumStrategy
  - BreakoutStrategy
  - ValueInvestingStrategy

- **Arbitrage agents** are appended following these (starting with agent ID 100).

We reconstruct the strategy for each agent using its ID.

In [ ]:
# Set parameters (should match the simulation run)
num_agents = 100  # agents with IDs 0-99

# Function to get the strategy name based on agent id
def get_strategy_name(agent_id):
    if agent_id < num_agents:
        strategies = ['TrendFollowing', 'MeanReversion', 'Momentum', 'Breakout', 'ValueInvesting']
        return strategies[agent_id % len(strategies)]
    else:
        return 'Arbitrage'

# Extract final wealth for each agent from the last tick
final_wealth = wealth_data.iloc[-1]
strategy_map = {int(agent): get_strategy_name(int(agent)) for agent in wealth_data.columns}

# Create a DataFrame for final wealth per agent along with their strategy
agent_performance = pd.DataFrame({
    'AgentID': [int(a) for a in wealth_data.columns],
    'FinalWealth': final_wealth.values,
    'Strategy': [strategy_map[int(a)] for a in wealth_data.columns]
})

print("Agent Performance (Final Wealth):")
agent_performance.head()

## Performance Statistics by Strategy

We now group the agents by their strategy and compute summary statistics: mean, median, standard deviation, minimum, and maximum of final wealth.

In [ ]:
performance_stats = agent_performance.groupby('Strategy')['FinalWealth'].agg(['mean', 'median', 'std', 'min', 'max'])
print("\nPerformance Statistics by Strategy:")
print(performance_stats)

## Overall Performance vs. Change in Price

The notebook now calculates the average wealth over time across all agents and compares it to the asset price. We then compute the percentage changes and a correlation between price changes and wealth changes.

In [ ]:
# Compute average agent wealth per tick (across all agents)
avg_wealth_over_time = wealth_data.mean(axis=1)

# Plot asset price vs. average agent wealth over time
ticks = price_data.index.astype(int)

plt.figure(figsize=(12,6))
plt.plot(ticks, price_data['Price'], marker='o', label='Asset Price')
plt.plot(ticks, avg_wealth_over_time, marker='x', label='Average Agent Wealth')
plt.xlabel('Tick')
plt.ylabel('Value')
plt.title('Asset Price vs. Average Agent Wealth Over Time')
plt.legend()
plt.show()

# Calculate percentage change in asset price between ticks
price_data['PriceChangePct'] = price_data['Price'].pct_change()

# Calculate percentage change in average wealth
avg_wealth_change = avg_wealth_over_time.pct_change()

# Compute correlation (drop the NaN values from percentage change computations)
correlation = np.corrcoef(price_data['PriceChangePct'].dropna(), avg_wealth_change.dropna())[0, 1]
print(f"Correlation between price change and average wealth change: {correlation:.4f}")

## Distribution of Final Wealth by Strategy

A box plot below shows the distribution of final wealth for each trading strategy.

In [ ]:
plt.figure(figsize=(10,6))
agent_performance.boxplot(column='FinalWealth', by='Strategy')
plt.title('Final Wealth Distribution by Trading Strategy')
plt.suptitle('')
plt.xlabel('Strategy')
plt.ylabel('Final Wealth')
plt.show()

## Summary

This analysis provided:

- Summary statistics for each trading strategy's final wealth
- A time-series comparison of the asset price and average agent wealth
- A computation of correlation between asset price changes and agent wealth changes
  
You can extend this analysis further by incorporating trade log details if you export that information from the simulation.